In [ ]:
%pip install unstructured
%pip install pinecone-client
%pip install tiktoken
%pip install langchain-community
%pip install sentence-transformers
%pip install chromadb
%pip install langchain langchain_core openai pinecone tqdm
%pip install "unstructured[pdf]"
%pip install torch
%pip install nltk

# optional
%pip install torch==2.0.0a0 intel_extension_for_pytorch==2.0.110+gitba7f6c1 -f https://developer.intel.com/ipex-whl-stable-xpu

In [15]:
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [16]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:
directory = './input_files'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

6

In [18]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

6270


In [19]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [20]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma")

In [26]:
query = "how to fuck machine learning"
matching_docs = db.similarity_search(query)
matching_docs[0]

Document(page_content='Outside of commercial applications, machine learning has had a tremendous influ‐ ence on the way data-driven research is done today. The tools introduced in this book have been applied to diverse scientific problems such as understanding stars, finding distant planets, discovering new particles, analyzing DNA sequences, and providing personalized cancer treatments.\n\nYour application doesn’t need to be as large-scale or world-changing as these exam‐ ples in order to benefit from machine learning, though. In this chapter, we will explain why machine learning has become so popular and discuss what kinds of problems can be solved using machine learning. Then, we will show you how to build your first machine learning model, introducing important concepts along the way.', metadata={'source': 'input_files\\Introduction to Machine Learning with Python ( PDFDrive.com )-min.pdf'})

In [27]:
from_db = matching_docs[0].page_content.replace("\t", " ").replace('\n\n', " ")

In [28]:
from_db

'Outside of commercial applications, machine learning has had a tremendous influ‐ ence on the way data-driven research is done today. The tools introduced in this book have been applied to diverse scientific problems such as understanding stars, finding distant planets, discovering new particles, analyzing DNA sequences, and providing personalized cancer treatments. Your application doesn’t need to be as large-scale or world-changing as these exam‐ ples in order to benefit from machine learning, though. In this chapter, we will explain why machine learning has become so popular and discuss what kinds of problems can be solved using machine learning. Then, we will show you how to build your first machine learning model, introducing important concepts along the way.'